In [ ]:
import os
# os.chdir('../')

## Export results for RBP-binding-effects and calculate lambda

In [ ]:
import pandas as pd
import numpy as np
import gzip
from glob import glob

In [ ]:
import scipy.stats as st

In [ ]:
from script.python.util.snake import clean_str

import yaml

with open('conf/config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:

if config['phenotypes'].endswith('.gz'):
    with gzip.open(config['phenotypes'],'rt') as f:
        phenotypes = f.readline().rstrip().split('\t')
else:
    with open(config['phenotypes'], 'r') as f:
        phenotypes = f.readline.rstrip().split('\t')

phenotypes = { clean_str(p): p  for p in phenotypes[1:] }

In [ ]:
def estimate_lambda(pv):

    '''
    estimate the lambda for a given array of P-values
    ------------------------------------------------------------------
    pv          numpy array containing the P-values
    ------------------------------------------------------------------
    L           lambda value
    ------------------------------------------------------------------
    '''
    
    LOD2 = np.median(st.chi2.isf(np.abs(pv[~np.isnan(pv)]), 1))
    L = (LOD2/st.chi2(1).median())
    
    return L

In [ ]:
def rename_cols(cols, suffix=''):
    return {c: c+suffix if c not in ['gene', 'pheno'] else c for c in cols}

In [ ]:
outfiles = glob('./work/association/sclrt_kernels_deepripe_multiple/all/*/results*')

In [ ]:
results = [ pd.read_csv(s, sep='\t', na_values='.', comment='#') for s in outfiles ]

In [ ]:
results = pd.concat(results)

In [ ]:
dropcols = list((c for c in results.columns if c.startswith('lrtstat_') or c.startswith('alteqnull_')))

In [ ]:
dropcols

In [ ]:
results = results[results.nCarrier >= 5]

In [ ]:
results.drop(columns=dropcols, inplace=True)

In [ ]:
pv_cols = list((c for c in results.columns if c.startswith('pv_')))

In [ ]:
pv_cols

In [ ]:
pv_cols = list((c for c in results.columns if c.startswith('pv')))

In [ ]:
results.columns

In [ ]:
results = results[ ['gene','nCarrier','nCarrier_notLOF','cumMAC','cumMAC_notLOF','n_snp','n_snp_notLOF'] + pv_cols + ['pheno'] ]

In [ ]:
results.rename(columns=rename_cols(results.columns, '_rbp'), inplace=True)

In [ ]:
results[['pheno']+results.columns.tolist()].sort_values('pv_lrt_linwcholesky_notLOF_rbp').head(20)

In [ ]:
tmp = results.copy()

pv_cols = list(c for c in tmp.columns if 'pv_' in c)
for col in pv_cols:
    if 'score' in col:
        if 'notLOF' not in col:
            continue
    if ('cLOF' in col) or ('mrgLOF' in col):
        continue
    kernel = col.split('_')[2]
    effect = col.split('_')[-1]
    tmp.loc[tmp[col].isna(),col] = tmp.loc[tmp[col].isna(), 'pv_score_{}_{}'.format(kernel, effect)]
    print(col)

results = tmp
del tmp

In [ ]:
lambda_val =  results.groupby(['pheno'])[pv_cols].agg(estimate_lambda)

In [ ]:
lambda_val

In [ ]:
lambda_val.to_csv('results/tables/lambdaval_rbp.tsv', sep='\t')
results.to_csv('results/tables/results_rbp.tsv.gz', sep='\t', index=False)